# Notebook: Analyse exploratoire et préparation de la modélisation


## 1. Installation et imports (Colab)

In [20]:
# Installation (dans Colab si besoin)
!pip install --quiet pandas numpy matplotlib scikit-learn shap imbalanced-learn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, precision_recall_curve, average_precision_score
from imblearn.over_sampling import SMOTE

# On clone le git
!git clone https://github.com/Davidouu/technova-p4.git
%cd technova-p4

## 2. Chargement des données

In [21]:
df_sirh    = pd.read_csv('data/extrait_sirh.csv')
df_eval    = pd.read_csv('data/extrait_eval.csv')
df_sondage = pd.read_csv('data/extrait_sondage.csv')

## 3. Inspection initiale

In [22]:
def inspect(df, name):
    print(f"=== {name} ===")
    print("Shape:", df.shape)
    display(df.head(5))
    display(df.info())
    display(df.describe(include='all').transpose())
    print("Nulls per column:")
    display(df.isnull().sum())
    print("Duplicate rows:", df.duplicated().sum())
    print("\n")

inspect(df_sirh, "SIRH")
inspect(df_eval, "Évaluations")
inspect(df_sondage, "Sondage")

print("SIRH employee IDs sample:", df_sirh['id_employee'].unique()[:10])
print("Évaluations possible keys:", df_eval.columns.tolist())
print("Sondage columns:", df_sondage.columns.tolist())

=== SIRH ===
Shape: (1470, 12)


,id_employee,age,genre,revenu_mensuel,statut_marital,departement,poste,nombre_experiences_precedentes,nombre_heures_travailless,annee_experience_totale,annees_dans_l_entreprise,annees_dans_le_poste_actuel
0,1,41,F,5993,Célibataire,Commercial,Cadre Commercial,8,80,8,6,4
1,2,49,M,5130,Marié(e),Consulting,Assistant de Direction,1,80,10,10,7
2,4,37,M,2090,Célibataire,Consulting,Consultant,6,80,7,0,0
3,5,33,F,2909,Marié(e),Consulting,Assistant de Direction,1,80,8,8,7
4,7,27,M,3468,Marié(e),Consulting,Consultant,9,80,6,2,2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 12 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   id_employee                     1470 non-null   int64 
 1   age                             1470 non-null   int64 
 2   genre                           1470 non-null   object
 3   revenu_mensuel                  1470 non-null   int64 
 4   statut_marital                  1470 non-null   object
 5   departement                     1470 non-null   object
 6   poste                           1470 non-null   object
 7   nombre_experiences_precedentes  1470 non-null   int64 
 8   nombre_heures_travailless       1470 non-null   int64 
 9   annee_experience_totale         1470 non-null   int64 
 10  annees_dans_l_entreprise        1470 non-null   int64 
 11  annees_dans_le_poste_actuel     1470 non-null   int64 
dtypes: int64(8), object(4)
memory usage: 137.9+ KB


None

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id_employee,1470.0,NaN,NaN,NaN,1024.865306,602.024335,1.0,491.25,1020.5,1555.75,2068.0
age,1470.0,NaN,NaN,NaN,36.92381,9.135373,18.0,30.0,36.0,43.0,60.0
genre,1470,2,M,882,NaN,NaN,NaN,NaN,NaN,NaN,NaN
revenu_mensuel,1470.0,NaN,NaN,NaN,6502.931293,4707.956783,1009.0,2911.0,4919.0,8379.0,19999.0
statut_marital,1470,3,Marié(e),673,NaN,NaN,NaN,NaN,NaN,NaN,NaN
departement,1470,3,Consulting,961,NaN,NaN,NaN,NaN,NaN,NaN,NaN
poste,1470,9,Cadre Commercial,326,NaN,NaN,NaN,NaN,NaN,NaN,NaN
nombre_experiences_precedentes,1470.0,NaN,NaN,NaN,2.693197,2.498009,0.0,1.0,2.0,4.0,9.0
nombre_heures_travailless,1470.0,NaN,NaN,NaN,80.0,0.0,80.0,80.0,80.0,80.0,80.0
annee_experience_totale,1470.0,NaN,NaN,NaN,11.279592,7.780782,0.0,6.0,10.0,15.0,40.0


...

None

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
a_quitte_l_entreprise,1470,2,Non,1233,NaN,NaN,NaN,NaN,NaN,NaN,NaN
nombre_participation_pee,1470.0,NaN,NaN,NaN,0.793878,0.852077,0.0,0.0,1.0,1.0,3.0
nb_formations_suivies,1470.0,NaN,NaN,NaN,2.79932,1.289271,0.0,2.0,3.0,3.0,6.0
nombre_employee_sous_responsabilite,1470.0,NaN,NaN,NaN,1.0,0.0,1.0,1.0,1.0,1.0,1.0
code_sondage,1470.0,NaN,NaN,NaN,1024.865306,602.024335,1.0,491.25,1020.5,1555.75,2068.0
distance_domicile_travail,1470.0,NaN,NaN,NaN,9.192517,8.106864,1.0,2.0,7.0,14.0,29.0
niveau_education,1470.0,NaN,NaN,NaN,2.912925,1.024165,1.0,2.0,3.0,4.0,5.0
domaine_etude,1470,6,Infra & Cloud,606,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ayant_enfants,1470,1,Y,1470,NaN,NaN,NaN,NaN,NaN,NaN,NaN
frequence_deplacement,1470,3,Occasionnel,1043,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Nulls per column:


,0
a_quitte_l_entreprise,0
nombre_participation_pee,0
nb_formations_suivies,0
nombre_employee_sous_responsabilite,0
code_sondage,0
distance_domicile_travail,0
niveau_education,0
domaine_etude,0
ayant_enfants,0
frequence_deplacement,0


Duplicate rows: 0


SIRH employee IDs sample: [ 1  2  4  5  7  8 10 11 12 13]
Évaluations possible keys: ['satisfaction_employee_environnement', 'note_evaluation_precedente', 'niveau_hierarchique_poste', 'satisfaction_employee_nature_travail', 'satisfaction_employee_equipe', 'satisfaction_employee_equilibre_pro_perso', 'eval_number', 'note_evaluation_actuelle', 'heure_supplementaires', 'augementation_salaire_precedente']
Sondage columns: ['a_quitte_l_entreprise', 'nombre_participation_pee', 'nb_formations_suivies', 'nombre_employee_sous_responsabilite', 'code_sondage', 'distance_domicile_travail', 'niveau_education', 'domaine_etude', 'ayant_enfants', 'frequence_deplacement', 'annees_depuis_la_derniere_promotion', 'annes_sous_responsable_actuel']


## 4. Extraction et nettoyage initial

In [23]:
# 1. Eval : extraire id_employee depuis eval_number

def extract_id_from_eval(x):
    if isinstance(x, str) and x.startswith("E_"):
        try:
            return int(x.split("_", 1)[1])
        except:
            return None
    return None

df_eval['id_employee'] = df_eval['eval_number'].apply(extract_id_from_eval)

# 2. Heures supplémentaires : binaire
df_eval['heure_supplementaires_flag'] = df_eval['heure_supplementaires'].str.lower().map({'oui': 1, 'non': 0})

# 3. Augmentation salaire précédente : pourcentage -> float
def parse_percent(x):
    if isinstance(x, str):
        return float(x.replace('%', '').strip()) / 100
    try:
        return float(x)
    except:
        return np.nan

df_eval['augmentation_salaire_precedente_pct'] = df_eval['augementation_salaire_precedente'].apply(parse_percent)

# 4. Sondage : encode la cible
df_sondage['a_quitte_flag'] = df_sondage['a_quitte_l_entreprise'].str.lower().map({'oui': 1, 'non': 0})

## 5. Jointures et construction du DataFrame central

In [24]:
df_merge = df_sirh.merge(df_eval, on='id_employee', how='inner', suffixes=('_sirh', '_eval'))
df_central = df_merge.merge(
    df_sondage.rename(columns={'code_sondage': 'id_employee'}),
    on='id_employee',
    how='inner',
    suffixes=('', '_sondage')
)

print("Shape du central:", df_central.shape)
print("Répartition de la démission :")
print(df_central['a_quitte_flag'].value_counts(normalize=True))

Shape du central: (1470, 36)
Répartition de la démission :
a_quitte_flag
0    0.838776
1    0.161224
Name: proportion, dtype: float64


## 6. Préparation de X et y (features et cible)

In [26]:
# --- Préparation propre de X et y ---

def clean_and_encode(df):
    df = df.copy()

    # Supprimer les colonnes brutes déjà transformées / redondantes
    for col in ['heure_supplementaires', 'augementation_salaire_precedente', 'a_quitte_l_entreprise', 'eval_number']:
        if col in df.columns:
            df = df.drop(columns=[col])

    # Ordinalisation
    freq_map = {'Occasionnel': 0, 'Frequent': 1}
    if 'frequence_deplacement' in df.columns:
        df['frequence_deplacement_ord'] = df['frequence_deplacement'].map(freq_map)

    # Encodage nominal (incluant ayant_enfants)
    cat_cols = [c for c in ['statut_marital', 'departement', 'poste', 'domaine_etude', 'ayant_enfants'] if c in df.columns]
    if cat_cols:
        df = pd.get_dummies(df, columns=cat_cols, drop_first=True)

    # Ratio : revenu par année
    if 'revenu_mensuel' in df.columns and 'annees_dans_l_entreprise' in df.columns:
        df['revenu_par_annee'] = df['revenu_mensuel'] / df['annees_dans_l_entreprise'].replace(0, np.nan)

    return df

def build_X_y(df_central):
    df_prepped = clean_and_encode(df_central)
    y = df_prepped['a_quitte_flag'].astype(int)
    X = df_prepped.drop(columns=['a_quitte_flag', 'id_employee'], errors='ignore')
    # Retirer toute colonne non numérique restante
    non_numeric = X.select_dtypes(exclude=['number']).columns.tolist()
    if non_numeric:
        print("Suppression de colonnes non numériques :", non_numeric)
        X = X.drop(columns=non_numeric)
    return X, y

# Construction initiale
X, y = build_X_y(df_central)

# Imputation des missing (avant toute sélection/corrélation)
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

print("Shape de X après imputation:", X.shape)
print("Répartition de y :", y.value_counts(normalize=True))

# Suppression des features fortement corrélées (Pearson)
def remove_high_corr(X, threshold=0.85):
    corr = X.corr().abs()
    upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
    to_drop = [col for col in upper.columns if any(upper[col] > threshold)]
    X_reduced = X.drop(columns=to_drop)
    return X_reduced, to_drop

X_reduced, dropped = remove_high_corr(X, threshold=0.85)
print("Features supprimées pour forte corrélation (Pearson > 0.85) :", dropped)

# Final
X_final = X_reduced.copy()
y_final = y.copy()


Suppression de colonnes non numériques : ['genre', 'frequence_deplacement', 'statut_marital_Divorcé(e)', 'statut_marital_Marié(e)', 'departement_Consulting', 'departement_Ressources Humaines', 'poste_Cadre Commercial', 'poste_Consultant', 'poste_Directeur Technique', 'poste_Manager', 'poste_Représentant Commercial', 'poste_Ressources Humaines', 'poste_Senior Manager', 'poste_Tech Lead', 'domaine_etude_Entrepreunariat', 'domaine_etude_Infra & Cloud', 'domaine_etude_Marketing', 'domaine_etude_Ressources Humaines', 'domaine_etude_Transformation Digitale']
Shape de X après imputation: (1470, 25)
Répartition de y : a_quitte_flag
0    0.838776
1    0.161224
Name: proportion, dtype: float64
Features supprimées pour forte corrélation (Pearson > 0.85) : ['niveau_hierarchique_poste']
